In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from first_stage_func import first_stage

In [ ]:
startdate = 0
enddate = 90
# autumn around 90 days, for simplicity

In [92]:
stations = pd.read_csv('datasets/station_with_demand.csv')

stations['current_bikes'] =0


In [93]:
model_results = first_stage()

print(model_results)

[99997.3199999999, 1.0, [36.0, 8.0, 0.0, 30.0, 3.0, 1.0, 33.0, 43.0, 0.0, 4.0, 8.0, 15.0, 7.0, 15.0, 3.0, 6.0, 53.0, 13.0, 14.0, 45.0, 46.0, 13.0, 2.0, 15.0, 10.0, 5.0, 14.0, 16.0, 32.0, 20.0, 8.0, 4.0, 0.0, 7.0, 14.0, 45.0, 44.0, 27.0, 2.0, 29.0, 68.0, 25.0, 1.0, 14.0, 23.0, 7.0, 31.0, 7.0, 13.0, 39.0, 20.0, 18.0, 0.0, 49.0, 1.0, 9.0, 24.0, 17.0, 0.0, 20.0, 2.0, 34.0, 4.0, 41.0, 11.0, 1.0, 19.0, 1.0, 2.0, 66.0, 10.0, 11.0, 4.0, 53.0, 17.0, 6.0, 3.0, 30.0, 37.0, 47.0, 0.0, 13.0, 17.0, 20.0, 34.0, 25.0, 12.0, 0.0, 66.0, 0.0, 18.0, 8.0, 0.0, 0.0, 1.0, 26.0, 26.0, 47.0, 11.0, 8.0, 27.0, 10.0, 0.0, 21.0, 20.0, 0.0, 0.0, 0.0, 10.0, 15.0, 22.0, 20.0, 0.0, 22.0, 12.0, 29.0, 26.0, 30.0, 0.0, 32.0, 2.0, 23.0, 11.0, 0.0, 5.0, 34.0, 6.0, 4.0, 10.0, 22.0, 28.0, 36.0, 34.0, 55.0, 0.0, 12.0, 19.0, 57.0, 5.0, 21.0, 48.0, 8.0, 56.0, 4.0, 2.0, 25.0, 12.0, 0.0, 0.0, 25.0, 48.0, 56.0, 54.0, 13.0, 23.0, 18.0, 0.0, 0.0, 8.0, 24.0, 3.0, 0.0, 14.0, 0.0, 24.0, 15.0, 0.0, 11.0, 26.0, 16.0, 8.0, 10.0, 32.0, 13.

In [94]:

demand_cat = ['Highest', 'Medium', 'Low', 'Lowest']

In [95]:
manhatten_bike_fleet = int(662/1915 * 37369)

In [ ]:
while startdate < enddate:
    
    bike_limit = manhatten_bike_fleet

    for i, row in stations.iterrows():
        
        random_factor = random.randint(0,1000)
        
        # Random current bikes in station
        limit = min(stations.iloc[i]['capacity'], bike_limit)
        currentBikes = random.randint(0, int(0.2*limit))
        stations.at[i,'current_bikes'] = currentBikes
        bike_limit -= currentBikes
        
        # Random demand of station
        current_demand_cat = demand_cat.index(stations.iloc[i]['DEMAND_CATEGORY'])
        if random_factor < 50 and current_demand_cat != 3:
            stations.at[i,'DEMAND_CATEGORY'] = demand_cat[current_demand_cat + 1]
            
        elif random_factor> 950 and current_demand_cat != 0:
            stations.at[i,'DEMAND_CATEGORY'] = demand_cat[current_demand_cat - 1]

    if bike_limit > 0:
        while bike_limit > 0:
            i = random.randint(0, len(stations) - 1)
            limit = min(stations.iloc[i]['capacity'], bike_limit) 
            current_random = stations.iloc[i]['current_bikes']
            current_available = limit - current_random
            if current_available <= 0:
                continue
            to_fill = random.randint(0, current_available)
            stations.at[i,'current_bikes'] += to_fill
            bike_limit -= to_fill
    
    if startdate % 7 == 0:
        # first_stage_code
        i =1 #filler
        
    #second stage code
    
    startdate += 1

startdate =0

In [97]:
stations


,station_id,station_name,capacity,lng,lat,MEAN_WEEKLY_TRIPS,NORMALISED_CAPACITY,DEMAND_SCORE,DEMAND_CATEGORY,current_bikes
0,5772.05,Morton St & Greenwich St,56,-74.008870,40.731150,0,0.441667,0.000000,Low,2
1,6560.15,Broadway & W 38 St,68,-73.987349,40.752973,511,0.541667,234.719333,Highest,68
2,7727.07,Amsterdam Ave & W 119 St,29,-73.959621,40.808625,0,0.216667,0.000000,Lowest,29
3,6569.08,W 35 St & Dyer Ave,41,-73.997402,40.754692,206,0.316667,140.972667,Lowest,41
4,7023.04,W 59 St & 10 Ave,117,-73.988038,40.770513,341,0.950000,17.391000,Low,96
...,...,...,...,...,...,...,...,...,...,...
657,8416.10,W 186 St & St Nicholas Ave,26,-73.931308,40.852253,8,0.191667,6.474667,Low,0
658,4846.01,South St & Whitehall St,55,-74.012342,40.701221,313,0.433333,177.679667,Highest,5
659,6890.01,W 53 St & 10 Ave,29,-73.990617,40.766697,424,0.216667,332.557333,Lowest,26
660,6483.06,W 36 St & 7 Ave,38,-73.989539,40.752149,173,0.291667,122.714667,Low,3
